### Install the required Python modules/SDKs 

In [2]:
! pip install azure-cognitiveservices-vision-customvision

     -------------------------------------- 62.0/62.0 kB 822.1 kB/s eta 0:00:00


## Import libraries

In [1]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os, time, uuid, glob

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


## Creating Training & Prediting Client with credentials

In [2]:
TRAINING_ENDPOINT = "https://southeastasia.api.cognitive.microsoft.com/"
training_key = "665348083c644194b2412cc6573cb764"
training_resource_id = '/subscriptions/66f1797a-2df0-402c-87dc-4ef25e3aeb7e/resourceGroups/aind-244110/providers/Microsoft.CognitiveServices/accounts/ducpc1_cv_training'

PREDICTION_ENDPOINT = 'https://southeastasia.api.cognitive.microsoft.com/'
prediction_key = "e0f33ca749034e8a8b36e4a64c8f320a"
prediction_resource_id = "/subscriptions/66f1797a-2df0-402c-87dc-4ef25e3aeb7e/resourceGroups/aind-244110/providers/Microsoft.CognitiveServices/accounts/ducpc1_cv_predict"

In [3]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)
trainer.api_version

'3.4-preview'

In [4]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)
predictor.api_version

'3.1'

## Get project ID and iteration ID from portal

In [5]:
project_id = trainer.get_projects()[0].id # Assume this is the only project on portal
print("Project ID: ", project_id)

Project ID:  a0c138ba-42fc-41e3-91b7-492b67f69ab0


In [8]:
iteration_list = trainer.get_iterations(project_id)
chosen_iteration_id = None
for iteration in iteration_list:
    if iteration.name == "Iteration 1":
        chosen_iteration_id = iteration.id

## Publish api with selected training iteration

In [9]:
publish_iteration_name = "lighter-detection-ducpc1"
trainer.publish_iteration(project_id, chosen_iteration_id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Perform prediction on test baggage images

In [16]:
def perform_prediction(predictor, publish_iteration_name, image_folder, threshold=0.8):
    list_images = glob.glob(os.path.join(image_folder, '*.jpg'))
    print(len(list_images))
    for image_file_path in list_images:
        print("Predicting image: ", image_file_path)
        with open(image_file_path, "rb") as image_contents:
            results = predictor.detect_image(project_id, publish_iteration_name, image_contents.read())
            # Display the results.
            for prediction in results.predictions:
                if prediction.probability >= threshold:
                    print("\t" + prediction.tag_name +
                          ": {0:.2f}%".format(prediction.probability * 100))

In [21]:
image_test = r"D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images"
perform_prediction(predictor, publish_iteration_name, image_folder=image_test)

6
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\lighter_test_set_1of5.jpg
	lighter: 81.25%
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\lighter_test_set_2of5.jpg
	lighter: 99.65%
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\lighter_test_set_3of5.jpg
	lighter: 95.28%
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\lighter_test_set_4of5.jpg
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\lighter_test_set_5of5.jpg
	lighter: 99.72%
	lighter: 98.01%
Predicting image:  D:\Working\Code\Udacity\Duc_Project\computer_vision\starter\lighter_test_images\paggage_no_lighter.jpg
	lighter: 98.46%
